# О задаче

* Работа с аннотациями учебных дисциплин
* Компьютерные науки
* Очна форма обучения
* Бакалавриат
* Просмотр и визуальный анализ данных
* Заполнение пропусков (недостающие ссылки, зачетные единицы и тд.)
* Расширение данных (новые ОП и тд)
* Обработка текстовых данных
* Кластеризация

In [1]:
import pandas as pd
import os

from IPython.display import display, clear_output
import time, webbrowser

# Знакомство с данными

In [2]:
df = pd.read_excel('Таблица ОП разных вузов.xlsx', sheet_name='Дисциплины')
df['id'] = df.index
df.head(2)

,ВУЗ,Факультет,Образовательная программа,Уровень,Дисциплина,Курс,Тип,Зачетые единицы,Ссылка на описание дисциплины,Комментарии,Unnamed: 10,Unnamed: 11,id
0,Алтайский государственный технический универси...,Факультет информационных технологий,Информатика и вычислительная техника,Бакалавриат,История,1,Обязательный,4.0,https://www.altstu.ru/omkofile/587681/name/Ann...,NaN,NaN,NaN,0
1,Алтайский государственный технический универси...,Факультет информационных технологий,Информатика и вычислительная техника,Бакалавриат,Философия,2,Обязательный,4.0,https://www.altstu.ru/omkofile/595128/name/Ann...,NaN,NaN,NaN,1


In [3]:
df.drop(columns=['Комментарии', 'Unnamed: 10', 'Unnamed: 11'], inplace=True)
df.rename(columns={'Образовательная программа':'ОП',
                  'Ссылка на описание дисциплины':'Ссылка',
                  'Зачетые единицы':'ЗЕ'}, inplace=True)
df.head(2)

,ВУЗ,Факультет,ОП,Уровень,Дисциплина,Курс,Тип,ЗЕ,Ссылка,id
0,Алтайский государственный технический универси...,Факультет информационных технологий,Информатика и вычислительная техника,Бакалавриат,История,1,Обязательный,4.0,https://www.altstu.ru/omkofile/587681/name/Ann...,0
1,Алтайский государственный технический универси...,Факультет информационных технологий,Информатика и вычислительная техника,Бакалавриат,Философия,2,Обязательный,4.0,https://www.altstu.ru/omkofile/595128/name/Ann...,1


In [4]:
df.isnull().sum(axis = 0)

ВУЗ              0
Факультет        0
ОП               0
Уровень          0
Дисциплина       0
Курс            35
Тип             31
ЗЕ             270
Ссылка        2358
id               0
dtype: int64

In [5]:
to_process = ['ВУЗ', 'Факультет', 'ОП', 'Уровень', 'Дисциплина', 'Тип']
# df.dropna(subset=to_process + ['Курс'], inplace=True)
df = df.fillna('')
df.isnull().sum(axis = 0)

ВУЗ           0
Факультет     0
ОП            0
Уровень       0
Дисциплина    0
Курс          0
Тип           0
ЗЕ            0
Ссылка        0
id            0
dtype: int64

In [6]:
def row_process(row):
    row_ = []
    for col in row:
        col = col.replace('\n', ' ').replace('\t', ' ')
        row_ += [col.lower()]
    return row_

df[to_process] = df.apply(lambda x: pd.Series(row_process(list(x[to_process])), 
                index=to_process), axis=1)

In [7]:
df['Тип'].unique()

array(['обязательный', 'по выбору', 'практика', 'вкр', '', 'по',
       'вариативный (обязательный)', 'вариативный (элективный)',
       'факультатив', 'вариативный', 'варитивный', 'дисциплина по выбору',
       'вариативная', 'элективный', 'вариативный (дисциплина профиля)',
       'вариативный (дисциплина по выбору(элективные))',
       'элективный модуль', 'практика (вариативная)', 'по выбору '],
      dtype=object)

In [8]:
df[(df['Тип'] == 'обязательный') | (df['Тип'] == 'вариативный (обязательный)') |
   (df['Тип'] == 'вариативный (дисциплина профиля)') | (df['Тип'] == 'практика') | (df['Тип'] == 'вкр')].shape

(3040, 10)

In [9]:
df[df['Тип'] == 'по']['Дисциплина'].tolist()

['введение в профессию (адаптивная дисциплина для лиц с ограниченными возможностями здоровья)']

In [10]:
general = ['обязательный']
elective = ['по выбору', 'по', 'вариативный (элективный)', 'вариативный', 'варитивный', 'дисциплина по выбору',
       'вариативная', 'элективный', 'вариативный (дисциплина по выбору(элективные))', 'элективный модуль', 'по выбору ']

other = ['факультатив', 'практика', 'вкр', 'вариативный (дисциплина профиля)', '']
def course_type(row):
    if row in general:
        return 'обязательный'
    if row in elective:
        return 'элективный'
    return row
    
df['Тип'] = df['Тип'].apply(course_type)
df.head(2)

,ВУЗ,Факультет,ОП,Уровень,Дисциплина,Курс,Тип,ЗЕ,Ссылка,id
0,алтайский государственный технический универси...,факультет информационных технологий,информатика и вычислительная техника,бакалавриат,история,1,обязательный,4.0,https://www.altstu.ru/omkofile/587681/name/Ann...,0
1,алтайский государственный технический универси...,факультет информационных технологий,информатика и вычислительная техника,бакалавриат,философия,2,обязательный,4.0,https://www.altstu.ru/omkofile/595128/name/Ann...,1


In [11]:
df['ВУЗ'].unique()

array(['алтайский государственный технический университет имени и.и. ползунова',
       'амурский государственный университет',
       'алтайский государственный университет',
       'адыгейский государственный университет',
       'академия маркетинга и социально-информационных технологий "имсит"',
       'волгоградский государственный университет',
       'волжский университет имени в.н. татищева',
       'астраханский государственный университет',
       'астраханский государственный технический университет (агту)',
       'байкальский государственный университет',
       'вятский социально-экономический институт',
       'башкирский государственный университет',
       'брянский государственный университет имени академика и. г. петровского',
       'белгородский государственный национальный исследовательский университет',
       'брянский государственный технический университет',
       'вологодский государственный университет',
       'волгоградский государственный технический уни

In [12]:
U_dict = {
    'алтайский государственный технический университет имени и.и. ползунова':'АлтГТУ',
    'амурский государственный университет':'АмГУ',
    'алтайский государственный университет':'АлтГУ',
    'адыгейский государственный университет':'АГУ',
    'академия маркетинга и социально-информационных технологий "имсит"':'ИМСИТ',
    'волгоградский государственный университет':'ВолГУ',
    'волжский университет имени в.н. татищева':'ВУиТ',
    'астраханский государственный университет':'АстГУ',
    'астраханский государственный технический университет (агту)':'АГТУ',
    'байкальский государственный университет':'БГУ',
    'вятский социально-экономический институт':'ВСЭИ',
    'башкирский государственный университет':'БашГУ',
    'брянский государственный университет имени академика и. г. петровского':'БГУ Петровского',
    'белгородский государственный национальный исследовательский университет':'БелГУ',
    'брянский государственный технический университет':'БГТУ',
    'вологодский государственный университет':'ВоГУ',
    'волгоградский государственный технический университет (волггту) ':'ВолгГТУ'
}

F_dict ={
    'факультет информационных технологий':'ФИТ',
    'факультет математики и информатики':'ФМиИ',
    'институт математики и информационных технологий':'ИМиИТ',
    'инженерно-физический факультет':'ИФФ',
    'институт информационных технологий и инноваций':'ИИТиИ',
    'институт цифровых технологий, электроники и физики':'ИЦТЭФ',
    'факультет математики и компьютерных наук':'ФМиКН',
    'факультет информатики и телекоммуникаций':'ФИиТ',
    'институт истории и международных отношений':'ИИиМО',
    'институт искусств и дизайна':'ИИиД',
    'международный институт экономики, менеджмента и информационных систем':'МИЭМиИ',
    'факультет цифровых технологий и кибербезопасности':'ФЦТиК',
    'институт информационных технологий и коммуникаций':'ИИТиК',
    'информационные системы и технологии в управлении':'ИСиТУ',
    'кафедра информатики и вычислительной техники':'КИиВТ',
    'факультет математики и информационных технологий':'ФМиИТ',
    'физико-математический факультет':'ФМФ',
    'факультет информационных технологий (фит)':'ФИТ',
    'институт математики, естественных и компьютерных наук':'ИМЕКН',
    'факультет электроники и вычислительной техники (фэвт)':'ФЭиВТ'
}

In [13]:
df['ВУЗ'] = df['ВУЗ'].map(U_dict)
df['Факультет'] = df['Факультет'].map(F_dict)
df.head(3)

,ВУЗ,Факультет,ОП,Уровень,Дисциплина,Курс,Тип,ЗЕ,Ссылка,id
0,АлтГТУ,ФИТ,информатика и вычислительная техника,бакалавриат,история,1,обязательный,4.0,https://www.altstu.ru/omkofile/587681/name/Ann...,0
1,АлтГТУ,ФИТ,информатика и вычислительная техника,бакалавриат,философия,2,обязательный,4.0,https://www.altstu.ru/omkofile/595128/name/Ann...,1
2,АлтГТУ,ФИТ,информатика и вычислительная техника,бакалавриат,экономика,3,обязательный,3.0,https://www.altstu.ru/omkofile/704811/name/Ann...,2


In [14]:
df.to_excel('full_table.xlsx', index=False, encoding='utf_8')

In [15]:
df[(df['ВУЗ'] == 'АлтГТУ') &
  (df['ОП']=='информатика и вычислительная техника') &
   (df['Курс']==4) & (df['Тип']=='обязательный')]['Дисциплина'].unique()

array(['основы  предпринимательской деятельности', 'правоведение',
       'современная научная картина мира'], dtype=object)

In [16]:
df[df['Ссылка'] != ''].shape

(2937, 10)

In [17]:
df['Тип'].unique()

array(['обязательный', 'элективный', 'практика', 'вкр', '',
       'вариативный (обязательный)', 'факультатив',
       'вариативный (дисциплина профиля)', 'практика (вариативная)'],
      dtype=object)

In [18]:
df[(df['Тип'] != 'практика') & (df['Тип'] != 'вкр') & (df['Тип'] != 'практика (вариативная)')].shape

(4983, 10)

In [19]:
df = df[(df['Тип'] != 'практика') & (df['Тип'] != 'вкр') & (df['Тип'] != 'практика (вариативная)')].copy()

In [20]:
try:
    os.mkdir('Вузы')
except:
    pass
for u in df['ВУЗ'].unique():
    df[df['ВУЗ'] == u].to_excel('Вузы\\' + u + '.xlsx', index=False, encoding='utf_8')

# Валидация и обобщенное структурирование ОП

In [21]:
links = {
    'АлтГТУ':['https://www.altstu.ru/structure/unit/ur/', 'https://www.altstu.ru/sveden/common/'], # смотреть по кафедрам
    'АмГУ':'https://www.amursu.ru/sveden/education/educationDocs/',
    'АлтГУ':'https://www.asu.ru/sveden/education/programs/#bakalavriat',
    'АГУ':'https://adygnet.ru/obrazovanie/opop-vo/',
    'ИМСИТ':['https://www.imsit.ru/sveden/education/programs/nachalynoe-obrazovanie-44.03.01.html', 'https://www.imsit.ru/sveden/education/eduaccred/',
             'https://www.imsit.ru/sveden/education/eduop/'],
    'ВолГУ':['https://volsu.ru/struct/institutes/', 'https://volsu.ru/sveden/education/pages/education_programs.php', 
            'https://volsu.ru/sveden/education/pages/documents.php', 'https://volsu.ru/Abitur/bachelor/perechen-programm.php'],
    'ВУиТ':['http://ved.vuit.ru/Plans/Default.aspx', 'http://vuit.ru/faculties/',
            'http://ved.vuit.ru/Dek/Default.aspx?mode=spets', 'http://vuit.ru/sveden/education/index.html#opop'],
    'АстГУ':['https://asu.edu.ru/obrazovatelinaya-deyatelinost/9234-rabochie-programmy-disciplin.html',
             'https://asu.edu.ru/obrazovatelinaya-deyatelinost/5082-osnovnye-obrazovatelnye-programmy.html'],
    'АГТУ':'https://www.astu.org/sveden/education',
    'БГУ':'http://bgu.ru/sveden/education/plan.aspx',
    'ВСЭИ':'http://www.vsei.ru/gum-napravleniya-podgotovki',
    'БашГУ':['https://bashedu.ru/sveden/', 'https://bashedu.ru/sveden/education/eduaccred/', 'https://bashedu.ru/sveden/education/eduop/'],
    'БГУ Петровского':'https://brgu.ru/sveden/education/', # раздел "реализуемые ОП"
    'БелГУ':['https://dekanat.bsu.edu.ru/blocks/bsu_nabor/nabor.php', 'http://abitur.bsu.edu.ru/abitur/sp_napr/03/'],
    'БГТУ':'https://www.tu-bryansk.ru/sveden/education/eduop/',
    'ВоГУ':'https://vogu35.ru/sveden/education',
    'ВолгГТУ':['https://www.vstu.ru/sveden/education/', 'https://www.vstu.ru/sveden/education/']
}

## Удаление (объединение) дубликатов
`дисциплина1` преподается на 1, 2, 3 курсе и имеет три записи в таблице -- делаем одну запись, где `дисциплина1.Курс == 1,2,3`

In [22]:
subset_=['Дисциплина', 'ОП', 'ЗЕ', 'Ссылка']

unis = dict.fromkeys(links.keys())
for key in unis:
    unis[key] = pd.read_excel('Вузы\\' + key + '.xlsx').fillna({'Курс':0, 'ЗЕ':0.0}).fillna('')
    unis[key]['Курс'] = unis[key]['Курс'].astype(str)
    tmp_u = unis[key]
    for prog in tmp_u['ОП'].unique():
        tmp_p = tmp_u[tmp_u['ОП'] == prog]
        series = tmp_p.duplicated(subset=subset_, keep=False)
        dupl_series = tmp_p.loc[series.loc[series == True].index]
        
        for course in dupl_series['Дисциплина'].unique():
            ids = dupl_series[dupl_series['Дисциплина'] == course].index.to_list()
            for cnum in range(1, len(ids)):
                unis[key].loc[ids[0], 'Курс'] += ',' + unis[key].loc[ids[cnum]]['Курс']
        unis[key].drop_duplicates(subset=subset_, keep='first', inplace=True)

## Проверка полноты таблиц (см. набор курсов в учебных планах)

In [23]:
courses_df = pd.read_excel('full_table.xlsx')
courses_df = courses_df.fillna({'Курс':0, 'ЗЕ':0.0}).fillna('')
inds = courses_df[(courses_df['ВУЗ'] == 'БГТУ') & (courses_df['ОП'] == 'системы автоматизированного проектирования')
                  | (courses_df['ВУЗ'] == 'ВСЭИ') | (courses_df['ВУЗ'] == 'ВоГУ') | (courses_df['ВУЗ'] == 'БГУ')].index
courses_df.drop(index=inds, inplace=True)
inds = courses_df[(courses_df['ВУЗ'] == 'АстГУ') & (courses_df['ОП'] == 'информационные системы и технологии')].index
courses_df.loc[inds, 'ОП'] = 'технологии разработки и администрирования информационных систем'
courses_df.sample(2)

,ВУЗ,Факультет,ОП,Уровень,Дисциплина,Курс,Тип,ЗЕ,Ссылка,id
2229,ИМСИТ,ИИТиИ,разработка автоматизированных систем обработки...,бакалавриат,иностранный язык,2,обязательный,12.0,https://www.imsit.ru/sveden/files/RPD_B1.B.03_...,2229
3896,БашГУ,ФМиИТ,преподавание математики и информатики,бакалавриат,научно-исследовательская работа,4,практика,3.0,https://bashedu.ru/sveden/files/01.03.01_Matem...,3896


In [24]:
indexes = courses_df[courses_df['ОП'] == 'информационная безопасность автоматизированных систем'].index
courses_df.loc[indexes, 'ОП'] = 'безопасность автоматизированных систем (в сфере профессиональной деятельности)'

In [25]:
indexes = courses_df[courses_df['ОП'] == 'системы программирования и компьютерные технологии'].index
courses_df.loc[indexes, 'ОП'] = 'системное программирование и компьютерные технологии'

In [26]:
programs = courses_df.groupby(['ОП'], as_index=False)[['ВУЗ', 'Факультет', 'ОП']].head(1000).drop_duplicates()
programs.sort_values(by=['ВУЗ', 'Факультет', 'ОП'], inplace=True, ignore_index=True)
programs

,ВУЗ,Факультет,ОП
0,АГТУ,ИИТиК,информатика и вычислительная техника
1,АГТУ,ИИТиК,информационная безопасность
2,АГТУ,ИИТиК,прикладная информатика
3,АГТУ,ИИТиК,программная инженерия
4,АГУ,ИФФ,автоматизированные системы обработки информаци...
...,...,...,...
64,ВолгГТУ,ФЭиВТ,"вычислительные машины, комплексы, системы и сети"
65,ИМСИТ,ИИТиИ,автоматизированные системы обработки информаци...
66,ИМСИТ,ИИТиИ,разработка автоматизированных систем обработки...
67,ИМСИТ,ИИТиИ,разработка и проектирование информационных сис...


In [27]:
len(programs['ВУЗ'].unique())

14

In [28]:
class ProgramLinks():
    def __init__(self):
        self.codes = []
        self.plans = []
        self.annots = []
        self.workprograms = []
        self.annots_types = []
        self.workprograms_types = []
        
    def append(self, code, plan, annot, workprograms, wp='link', an='file'):
        self.codes.append(code)
        self.plans.append(plan)
        self.annots.append(annot)
        self.workprograms.append(workprograms)
        self.annots_types.append(an)
        self.workprograms_types.append(wp)

In [29]:
programLinks = ProgramLinks()

In [30]:
programs['ВУЗ'].unique()

array(['АГТУ', 'АГУ', 'АлтГТУ', 'АлтГУ', 'АмГУ', 'АстГУ', 'БГТУ',
       'БГУ Петровского', 'БашГУ', 'БелГУ', 'ВУиТ', 'ВолГУ', 'ВолгГТУ',
       'ИМСИТ'], dtype=object)

In [31]:
u = 'АлтГУ'
display(programs[programs['ВУЗ'] == u])
print(links[u]) if type(links[u]) != list else print(*links[u])

,ВУЗ,Факультет,ОП
14,АлтГУ,ИИиД,прикладная информатика в дизайне
15,АлтГУ,ИИиМО,прикладная информатика в гуманитарных науках
16,АлтГУ,ИМиИТ,математические основы компьютерных наук
17,АлтГУ,ИМиИТ,интеллектуальный анализ данных
18,АлтГУ,ИМиИТ,информатика и вычислительная техника
19,АлтГУ,ИМиИТ,математика и компьютерные науки
20,АлтГУ,ИМиИТ,математическое моделирование и информационные ...
21,АлтГУ,ИМиИТ,программирование и информационные технологии
22,АлтГУ,ИМиИТ,фундаментальная информатика и информационные т...
23,АлтГУ,ИЦТЭФ,безопасность автоматизированных систем (в сфер...


https://www.asu.ru/sveden/education/programs/#bakalavriat


In [32]:
# programLinks.append('', '',
#                     '', '',
#                     an='', wp='')

In [33]:
# АГТУ
programLinks.append('09.03.01', 'https://www.astu.org/Uploads/Sveden/004767.pdf',
                    'https://www.astu.org/Uploads/Sveden/006417.pdf',
                    'https://www.astu.org/Uploads/Sveden/006754.pdf', wp='file')

programLinks.append('10.03.01', 'https://www.astu.org/Uploads/Sveden/006436.pdf',
                    'https://www.astu.org/Uploads/Sveden/006382.pdf',
                    '')

programLinks.append('09.03.03', 'https://www.astu.org/Uploads/Sveden/006432.pdf',
                    'https://www.astu.org/Uploads/Sveden/006799.pdf',
                    'https://www.astu.org/Uploads/Sveden/006794.pdf', wp='file')

programLinks.append('09.03.04', 'https://www.astu.org/Uploads/Sveden/006434.pdf',
                    'https://www.astu.org/Uploads/Sveden/006401.pdf',
                    'https://www.astu.org/Uploads/Sveden/006808.pdf', wp='file')

# АГУ
programLinks.append('09.03.01', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/iff/ucheb_plan_IVT1.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/iff/Annot_IVT.pdf', '')
programLinks.append('10.03.01', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/matfak/Ucheb_plan_IB_2017.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop/matfak/new3/Annot_Disc_IB_Date.pdf', '')
programLinks.append('01.03.01', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/matfak/Ucheb_plan_M_Mat_Mod_2017.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop/matfak/new1/Annot_Disc_M_Mat_Mod_Date.pdf', '')
programLinks.append('01.03.01', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/matfak/Ucheb_plan_M_Prep_2017.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop/matfak/new1/Annot_Disc_M_Prep_Date.pdf', '')
programLinks.append('01.03.02', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/matfak/Ucheb_plan_PM_Sis_Prog_2017.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop/matfak/new1/Annot_Disc_PM_Sis_Prog_Date.pdf', '')
programLinks.append('02.03.03', 'https://adygnet.ru/obrazovanie/opop-vo/docs/oop_2018/matfak/Ucheb_plan_MOAIS_2017.pdf',
                    'https://adygnet.ru/obrazovanie/opop-vo/docs/oop/matfak/new1/Annot_Disc_MOAIS_Date.pdf', '')

# АлтГТУ
programLinks.append('09.03.01', 'https://www.altstu.ru/omkofile/uchplan/8225/name/Ucheb_plan_IVT_2018_28.05.2018.pdf',
                        'https://www.altstu.ru/sveden/oop/0524/', 'https://www.altstu.ru/sveden/oop/0524/',
                   an='link/html', wp='link/html')
programLinks.append('10.03.01', 'https://www.altstu.ru/omkofile/uchplan/8228/name/Ucheb_plan_IB_2018_28.05.2018.pdf',
                        'https://www.altstu.ru/sveden/oop/0532/', 'https://www.altstu.ru/sveden/oop/0532/',
                   an='link/html', wp='link/html')
programLinks.append('09.03.03', 'https://www.altstu.ru/omkofile/uchplan/8249/name/Ucheb_plan_PIE_2018_28.05.2018.pdf',
                        'https://www.altstu.ru/sveden/oop/0528/', 'https://www.altstu.ru/sveden/oop/0528/',
                   an='link/html', wp='link/html')
programLinks.append('09.03.04', 'https://www.altstu.ru/omkofile/uchplan/8240/name/Ucheb_plan_PI_2018_28.05.2018.pdf',
                        'https://www.altstu.ru/sveden/oop/0531/', 'https://www.altstu.ru/sveden/oop/0531/',
                   an='link/html', wp='link/html')

# АлтГУ
programLinks.append('09.03.03', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_03_prikladnaya%20informatika_pivd-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/1936/2021-33354/', 'https://www.asu.ru/sveden/education/programs/1936/2021-33354/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.03', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_03_pivgn-4-2021_2018_30.06.2020.pdf',
                    'https://www.asu.ru/sveden/education/programs/1151/2018-32790/', 'https://www.asu.ru/sveden/education/programs/1151/2018-32790/',
                    an='link/html', wp='link/html')
programLinks.append('02.03.01', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_02_03_01_matematika%20i%20kompyuternye%20nauki_mokn-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/2096/2021-32628/', 'https://www.asu.ru/sveden/education/programs/2096/2021-32628/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.03', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_03_prikladnaya%20informatika_iad-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/2106/2021-32641/', 'https://www.asu.ru/sveden/education/programs/2106/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.01', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_01_iivt-2-2021_2020_30.06.2020.pdf',
                    'https://www.asu.ru/sveden/education/programs/1413/2020-33016/', 'https://www.asu.ru/sveden/education/programs/1413/2020-33016/',
                    an='link/html', wp='link/html')
programLinks.append('02.03.01', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_02_03_01_mikn-4-2021_2018_30.06.2020.pdf',
                    'https://www.asu.ru/sveden/education/programs/880/2018-32631/', 'https://www.asu.ru/sveden/education/programs/880/2018-32631/',
                    an='link/html', wp='link/html')
programLinks.append('01.03.02', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_01_03_02_prikladnaya%20matematika%20i%20informatika_mmiit-1_2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/1424/2021-32623/', 'https://www.asu.ru/sveden/education/programs/1424/2021-32623/',
                    an='link/html', wp='link/html')
programLinks.append('02.03.02', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_02_03_02_fundamentalnaya%20informatika%20i%20informatsionnye%20tekhnologii_piit-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/2097/2021-32635/', 'https://www.asu.ru/sveden/education/programs/2097/2021-32635/',
                    an='link/html', wp='link/html')
programLinks.append('02.03.02', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_02_03_02_fiiit-4-2021_2018_30.06.2020.pdf',
                    'https://www.asu.ru/sveden/education/programs/882/2018-32634/', 'https://www.asu.ru/sveden/education/programs/882/2018-32634/',
                    an='link/html', wp='link/html')
programLinks.append('10.03.01', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_10_03_01_informatsionnaya%20bezopasnost_bas-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/1356/2021-33025/', 'https://www.asu.ru/sveden/education/programs/1356/2021-33025/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.03', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_03_pie-3-2021_2019_30.06.2020.pdf',
                    'https://www.asu.ru/sveden/education/programs/1423/2019-32655/', 'https://www.asu.ru/sveden/education/programs/1423/2019-32655/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.03', 'https://www.asu.ru/files/sveden/education/plan/Ucheb_plan_09_03_03_prikladnaya%20informatika_·еe-1-2021_2021_27.04.2021.pdf',
                    'https://www.asu.ru/sveden/education/programs/1449/2021-33355/', 'https://www.asu.ru/sveden/education/programs/1449/2021-33355/',
                    an='link/html', wp='link/html')

# АмГУ
programLinks.append('09.03.01', 'https://cabinet.amursu.ru/uploads/sveden/edu_prog/Ucheb_plan_Code_NN_Date/44997/Ucheb_plan_09.03.01_NN_10.06.2019.pdf',
                    'https://www.amursu.ru/sveden/education/educationDocs/', 'https://www.amursu.ru/sveden/education/educationDocs/',
                    an='link/html', wp='link/html')
programLinks.append('09.03.02', 'https://cabinet.amursu.ru/uploads/sveden/edu_prog/Ucheb_plan_Code_NN_Date/15916/Ucheb_plan_09.03.02_NN_30.10.2018.pdf',
                    'https://www.amursu.ru/sveden/education/educationDocs/', 'https://www.amursu.ru/sveden/education/educationDocs/',
                    an='link/html', wp='link/html')
programLinks.append('01.03.02', 'https://cabinet.amursu.ru/uploads/sveden/edu_prog/Ucheb_plan_Code_NN_Date/51255/Ucheb_plan_01.03.02_NN_28.11.2019.pdf',
                    'https://www.amursu.ru/sveden/education/educationDocs/', 'https://www.amursu.ru/sveden/education/educationDocs/',
                    an='link/html', wp='link/html')

# АстГУ
programLinks.append('09.03.01', 'https://asu.edu.ru/images/File/umu_oop/09_03_01_4_o_pr2_up_2021.pdf',
                    '', 'https://asu.edu.ru/obrazovatelinaya-deyatelinost/9234-rabochie-programmy-disciplin.html',
                    an='', wp='complex')
programLinks.append('10.03.01', 'https://asu.edu.ru/images/File/umu_oop/10_03_01_4_o_pr2_up_2021.pdf',
                    '', 'https://asu.edu.ru/obrazovatelinaya-deyatelinost/9234-rabochie-programmy-disciplin.html',
                    an='', wp='complex')
programLinks.append('09.03.02', 'https://asu.edu.ru/images/File/umu_oop/09_03_02_4_o_pr3_up_2021.pdf',
                    '', 'https://asu.edu.ru/obrazovatelinaya-deyatelinost/9234-rabochie-programmy-disciplin.html',
                    an='', wp='complex')
programLinks.append('01.03.02', 'https://asu.edu.ru/images/File/umu_oop/01_03_02_4_o_pr1_up_2021.pdf',
                    '', 'https://asu.edu.ru/obrazovatelinaya-deyatelinost/9234-rabochie-programmy-disciplin.html',
                    an='', wp='complex')

# БГТУ
programLinks.append('09.03.02', 'https://www.tu-bryansk.ru/upload/medialibrary/4ce/09.03.02_ИСТ_ИСиТвД_УП_4_2019_очная.pdf',
                    '', 'https://www.tu-bryansk.ru/upload/medialibrary/628/09.03.02_ИСТ_ИСиТвД_РПД_2018_ОЧН.zip',
                    an='', wp='zip')
programLinks.append('09.03.02', 'https://www.tu-bryansk.ru/upload/medialibrary/660/09.03.02%20ИСТ_ИТПК_очн.pdf',
                    'https://www.tu-bryansk.ru/upload/medialibrary/d8e/09.03.02_ИСТ_ИТиПК_АНН_2018_ОЧН.zip', 'https://www.tu-bryansk.ru/upload/medialibrary/897/09.03.02_ИСТ_ИТиПК_РПД_2018_ОЧН.zip',
                    an='zip', wp='zip')
programLinks.append('09.03.01', 'https://www.tu-bryansk.ru/upload/medialibrary/1f7/09.03.01%20ИВТ_ПО_очн.pdf',
                    'https://www.tu-bryansk.ru/upload/medialibrary/a42/09.03.01_ИВТ_ПО_Анн_2018_очная.zip', 'https://www.tu-bryansk.ru/upload/medialibrary/c61/09.03.01_ИВТ_ПО_РПД_2018_очная.rar',
                    an='zip', wp='rar')
programLinks.append('09.03.04', 'https://www.tu-bryansk.ru/upload/medialibrary/e34/09.03.04%20ПИ_РПИС_очн.pdf',
                    'https://www.tu-bryansk.ru/upload/medialibrary/7de/09.03.04_ПРИ_РПИС_Анн_2018_очная.zip', 'https://www.tu-bryansk.ru/upload/medialibrary/7ea/09.03.04_ПРИ_РПИС_РПД_2018_очная.zip',
                    an='zip', wp='zip')
programLinks.append('02.03.03', 'https://www.tu-bryansk.ru/upload/medialibrary/b85/02.03.03%20МОА_ТП_очн.pdf',
                    'https://www.tu-bryansk.ru/upload/medialibrary/9c3/02.03.03_МОА_ТП_Анн_2018_очная.zip', 'https://www.tu-bryansk.ru/upload/medialibrary/685/02.03.03_МОА_ТП_РПД_2018_очная.rar',
                    an='zip', wp='rar')

# БГУ Петровского
programLinks.append('02.03.02', 'https://brgu.ru/sveden/education/docs/opop/fmf/02.03.02_FIiIT/ochnay/2021/UP_FIiIT_2021.pdf',
                    'https://brgu.ru/sveden/education/docs/opop/fmf/02.03.02_FIiIT/ochnay/2021/AN_FIiIT_2021.pdf', 'https://eso.brgu.ru/mod/folder/view.php?id=372756',
                    an='file', wp='portal/link/html')
programLinks.append('01.03.02', 'https://brgu.ru/sveden/education/docs/opop/fmf/01.03.02_PMiI/2021/UP_PMiI_2021.pdf',
                    'https://brgu.ru/sveden/education/docs/opop/fmf/01.03.02_PMiI/2021/AN_PMiI_2021.pdf', 'https://brgu.ru/sveden/education/docs/opop/fmf/01.03.02_PMiI/2021/AN_PMiI_2021.pdf',
                    an='file', wp='file')

# БашГУ
programLinks.append('01.04.01', 'https://bashedu.ru/sveden/files/UP_01.03.01_Matematika_(Veschestvennyy,_kompleksnyy_i_funkcionalynyy_analiz)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_01.04.01_Matematika_(Veschestvennyy,_kompleksnyy_i_funkcionalynyy_analiz)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('01.04.01', 'https://bashedu.ru/sveden/files/UP_DO_01.04.01_Matematika_(Differencialynye_uravneniya,_dinamicheskie_sistemy_i_optimalynoe_upravlenie)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_01.04.01_Matematika_(Differencialynye_uravneniya,_dinamicheskie_sistemy_i_optimalynoe_upravlenie)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('09.03.03', 'https://bashedu.ru/sveden/files/UP_ZO_09.03.03_Prikladnaya_informatika_(Informacionnye_i_vychislitelynye_texnologii)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_09.03.03_Prikladnaya_informatika_(Informacionnye_i_vychislitelynye_texnologii)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('', 'https://bashedu.ru/sveden/files/01.03.02_PMI_(KIM)_2018.pdf',
                    '', '',
                    an='', wp='')
programLinks.append('01.03.02', 'https://bashedu.ru/sveden/files/UP_01.03.02_Prikladnaya_matematika_i_informatika_(Matematicheskoe_modelirovanie,_chislennye_metody_i_kompleksy_programm)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_01.03.02_Prikladnaya_matematika_i_informatika_(Matematicheskoe_modelirovanie,_chislennye_metody_i_kompleksy_programm)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('01.03.01', 'https://bashedu.ru/sveden/files/UP_01.03.01_Matematika_(Prepodavanie_matematiki_i_informatiki)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_01.03.01_Matematika_(Prepodavanie_matematiki_i_informatiki)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('02.03.03', 'https://bashedu.ru/sveden/files/UP_02.03.03_Matematicheskoe_obespechenie_i_administrirovanie_informacionnyx_sistem_(Sistemnoe_i_internet-programmirovanie)_2021.pdf',
                    'https://bashedu.ru/sveden/files/Annotacii_02.03.03_Matematicheskoe_obespechenie_i_administrirovanie_informacionnyx_sistem_(Sistemnoe_i_internet-programmirovanie)_2021.pdf',
                    '',
                    an='file', wp='')
programLinks.append('01.03.02', 'https://bashedu.ru/sveden/files/b01.03.02(SP,KT)_18_PMI.pdf',
                    'https://bashedu.ru/sveden/files/01.03.02_(Sistemnoe_programmirovanie_i_kompyyuternye_texnologii),_2018.pdf', '',
                    an='file', wp='')

# БелГУ
programLinks.append('09.03.02', 'https://dekanat.bsu.edu.ru/f.php/1/programmi/up/8855_up_37002_09.03.02_Informacionnye_sistemy_i_texnologii2021.xls',
                    '', 'https://dekanat.bsu.edu.ru/blocks/bsu_nabor/disciplines.php?sid=6387&kid=2&fid=2&yid=2021&pids=8855&tab=1&tab2=1&profileid=2629&facid=11200',
                    an='', wp='link')
programLinks.append('01.03.02', 'https://dekanat.bsu.edu.ru/f.php/1/programmi/up/8953_up_36998_01.03.02_Priladnaya_matematika_i_informatika2021.xls',
                    '', 'https://dekanat.bsu.edu.ru/blocks/bsu_nabor/disciplines.php?sid=6370&kid=2&fid=2&yid=2021&pids=8953&profileid=2792&facid=11200',
                    an='', wp='link')
programLinks.append('02.03.03', 'https://dekanat.bsu.edu.ru/f.php/1/programmi/up/8790_up_37000_02.03.03_Matem.obespechenie_i_admIS2021.plx.xls',
                    '', 'https://dekanat.bsu.edu.ru/blocks/bsu_nabor/disciplines.php?sid=6374&kid=2&fid=2&yid=2021&pids=8790&profileid=2595&facid=11200',
                    an='', wp='link')
programLinks.append('09.03.03', 'https://dekanat.bsu.edu.ru/f.php/1/programmi/up/8728_up_37003_09.03.03_Prikladnaya_informatika2021.plx.xls',
                    '', 'https://dekanat.bsu.edu.ru/blocks/bsu_nabor/disciplines.php?sid=6388&kid=2&fid=2&yid=2021&pids=8728,8728&profileid=2597&facid=11200',
                    an='', wp='link')
programLinks.append('02.03.02', 'https://dekanat.bsu.edu.ru/f.php/1/programmi/up/8789_up_36999_02.03.02_Fun.informatika_i_inf.texnologii2021.plx.xls',
                    '', 'https://dekanat.bsu.edu.ru/blocks/bsu_nabor/disciplines.php?sid=6373&kid=2&fid=2&yid=2021&pids=8789&profileid=2594&facid=11200',
                    an='', wp='link')

# ВУиТ
programLinks.append('09.03.01', 'http://vuit.ru/sveden/files/000678.pdf',
                    'http://vuit.ru/sveden/files/000113.pdf', 'https://web.archive.org/web/20210614015748/https://vuit.ru/sveden/education/rp/09.03.02/',
                    an='file', wp='link/html')
programLinks.append('09.03.02', 'http://vuit.ru/sveden/files/000682.pdf',
                    'http://vuit.ru/sveden/files/000122.pdf', 'https://web.archive.org/web/20170520155658/http://vuit.ru/faculties/it/files/section.php?SECTION_ID=546',
                    an='file', wp='link/html')

# ВолГУ
programLinks.append('10.03.01', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005420&typeFile=Учебный_план&name=УП%2010.03.01%20Информационная%20безопасность_Безопасность%20компьютерных%20систем_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('09.03.03', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005418&typeFile=Учебный_план&name=УП%2009.03.03%20Прикладная%20информатика_Информационное%20обеспечение%20автоматизированных%20систем_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('09.03.01', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005416&typeFile=Учебный_план&name=УП%2009.03.01%20Информатика%20и%20вычислительная%20техника_Компьютерное%20моделирование_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('01.03.02', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005456&typeFile=Учебный_план&name=УП%2001.04.02%20Прикладная%20математика%20и%20информатика_Математические%20методы%20обработки%20информации%20и%20принятия%20решений_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('02.03.01', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005408&typeFile=Учебный_план&name=УП%2002.03.01%20Математика%20и%20компьютерные%20науки_Математический%20анализ%20и%20компьютерные%20науки_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('09.03.02', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005417&typeFile=Учебный_план&name=УП%2009.03.02%20Информационные%20системы%20и%20технологии_Разработка%20информационных%20систем_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('09.03.04', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ004692&typeFile=Учебный_план&name=УП%2009.03.04%20Программная%20инженерия_Разработка%20программно-информационных%20систем_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')
programLinks.append('02.03.03', 'https://volsu.ru/sveden/education/pages/getFile.php?codeUP=АРМ005409&typeFile=Учебный_план&name=УП%2002.03.03%20Математическое%20обеспечение%20и%20администрирование%20информационных%20систем_Технология%20программирования_О',
                    'https://volsu.ru/sveden/education/pages/documents.php', '',
                    an='link/html', wp='')

# ВолгГТУ
programLinks.append('09.03.01', 'https://www.vstu.ru/upload/sveden/education/Ucheb_plan_09.03.01_A_CPh_O_NOR_FEVT_SAPR_2019.pdf',
                    'https://www.vstu.ru/upload/sveden/education/Annot_09.03.01_A_CPh_O_NOR_FEVT_SAPR_2019.pdf',
                    'https://www.vstu.ru/upload/sveden/education/Annot_09.03.01_A_CPh_O_NOR_FEVT_SAPR_2019.pdf',
                    an='file', wp='file')
programLinks.append('09.03.01', 'https://www.vstu.ru/upload/sveden/education/Ucheb_plan_09.03.01_A_VMKSiS_O_NOR_FEVT_EVM_2019.pdf',
                    'https://www.vstu.ru/upload/sveden/education/Annot_09.03.01_A_VMKSiS_O_NOR_FEVT_EVM_2015.pdf',
                    'https://www.vstu.ru/upload/sveden/education/Annot_09.03.01_A_VMKSiS_O_NOR_FEVT_EVM_2015.pdf',
                    an='file', wp='file')

# ИМСИТ
programLinks.append('09.03.01', 'https://www.imsit.ru/sveden/files/09.03.01_IVT_3++21.plx.pdf',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-avtomatizirovannyh-sistem-obrabotki-informacii-i-upravleniya-3-09.03.01.html',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-avtomatizirovannyh-sistem-obrabotki-informacii-i-upravleniya-3-09.03.01.html',
                    an='link/html', wp='link/html')

programLinks.append('09.03.01', 'https://www.imsit.ru/sveden/files/09.03.01_IVT_3++21.plx.pdf',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-avtomatizirovannyh-sistem-obrabotki-informacii-i-upravleniya-3-09.03.01.html',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-avtomatizirovannyh-sistem-obrabotki-informacii-i-upravleniya-3-09.03.01.html',
                    an='link/html', wp='link/html')

programLinks.append('09.03.03', 'https://www.imsit.ru/sveden/files/09.03.03_PI_3++21.plx.pdf',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-i-proektirovanie-informacionnyh-sistem-3-09.03.03.html',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-i-proektirovanie-informacionnyh-sistem-3-09.03.03.html',
                    an='link/html', wp='link/html')

programLinks.append('09.03.04', 'https://www.imsit.ru/sveden/files/09.03.04_PrIngh_3++21.plx.pdf',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-informacionno-vychislitelynyh-sistem-3-09.03.04.html',
                    'https://www.imsit.ru/sveden/education/programs/razrabotka-informacionno-vychislitelynyh-sistem-3-09.03.04.html',
                    an='link/html', wp='link/html')

In [34]:
programs['Код'] = programLinks.codes
programs['План'] = programLinks.plans
programs['Аннотация'] = programLinks.annots
programs['Дисциплина'] = programLinks.workprograms
programs['ан_тип'] = programLinks.annots_types
programs['рп_тип'] = programLinks.workprograms_types

programs.to_excel('programs.xlsx', index=False, encoding='utf_8')

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html

In [35]:
programs.loc[21, 'ОП'] = 'автоматизированные системы обработки информации и управления'
programs.loc[22, 'ОП'] = 'безопасность информационных систем'
display(programs.loc[21], programs.loc[22])

ВУЗ                                                       АлтГУ
Факультет                                                 ИМиИТ
ОП            автоматизированные системы обработки информаци...
Код                                                    02.03.02
План          https://www.asu.ru/files/sveden/education/plan...
Аннотация     https://www.asu.ru/sveden/education/programs/2...
Дисциплина    https://www.asu.ru/sveden/education/programs/2...
ан_тип                                                link/html
рп_тип                                                link/html
Name: 21, dtype: object

ВУЗ                                                       АлтГУ
Факультет                                                 ИМиИТ
ОП                           безопасность информационных систем
Код                                                    02.03.02
План          https://www.asu.ru/files/sveden/education/plan...
Аннотация     https://www.asu.ru/sveden/education/programs/8...
Дисциплина    https://www.asu.ru/sveden/education/programs/8...
ан_тип                                                link/html
рп_тип                                                link/html
Name: 22, dtype: object

In [36]:
len(programs[programs['Аннотация'] != '']), len(programs)

(58, 69)

In [37]:
programs[programs['Аннотация'] == '']

,ВУЗ,Факультет,ОП,Код,План,Аннотация,Дисциплина,ан_тип,рп_тип
29,АстГУ,ФЦТиК,информатика и вычислительная техника,09.03.01,https://asu.edu.ru/images/File/umu_oop/09_03_0...,,https://asu.edu.ru/obrazovatelinaya-deyatelino...,,complex
30,АстГУ,ФЦТиК,информационная безопасность,10.03.01,https://asu.edu.ru/images/File/umu_oop/10_03_0...,,https://asu.edu.ru/obrazovatelinaya-deyatelino...,,complex
31,АстГУ,ФЦТиК,прикладная математика и информатика,09.03.02,https://asu.edu.ru/images/File/umu_oop/09_03_0...,,https://asu.edu.ru/obrazovatelinaya-deyatelino...,,complex
32,АстГУ,ФЦТиК,технологии разработки и администрирования инфо...,01.03.02,https://asu.edu.ru/images/File/umu_oop/01_03_0...,,https://asu.edu.ru/obrazovatelinaya-deyatelino...,,complex
33,БГТУ,ФИТ,информационные системы и технологии в дизайне,09.03.02,https://www.tu-bryansk.ru/upload/medialibrary/...,,https://www.tu-bryansk.ru/upload/medialibrary/...,,zip
43,БашГУ,ФМиИТ,компьютерный инжиниринг и механика,,https://bashedu.ru/sveden/files/01.03.02_PMI_(...,,,,
48,БелГУ,ФМиИ,информационно-управляющие системы,09.03.02,https://dekanat.bsu.edu.ru/f.php/1/programmi/u...,,https://dekanat.bsu.edu.ru/blocks/bsu_nabor/di...,,link
49,БелГУ,ФМиИ,математическое моделирование и компьютерные те...,01.03.02,https://dekanat.bsu.edu.ru/f.php/1/programmi/u...,,https://dekanat.bsu.edu.ru/blocks/bsu_nabor/di...,,link
50,БелГУ,ФМиИ,параллельное программирование,02.03.03,https://dekanat.bsu.edu.ru/f.php/1/programmi/u...,,https://dekanat.bsu.edu.ru/blocks/bsu_nabor/di...,,link
51,БелГУ,ФМиИ,прикладная информатика в экономике,09.03.03,https://dekanat.bsu.edu.ru/f.php/1/programmi/u...,,https://dekanat.bsu.edu.ru/blocks/bsu_nabor/di...,,link


In [38]:
len(programs[programs['ан_тип'] == 'file'])

23

<!-- ## Просмотр пропущенных значений (ссылок) -->
<!-- См. `data_processing v0.ipynb` и соответствующие `txt`-файлы. -->

In [39]:
for ind, row in programs.iterrows():
    path = 'files/' + row['ВУЗ'] + '/' + row['Факультет'] + '_' + row['ОП'] + '/'
    try:
        os.makedirs(path)
    except:
        print('Already exists for ', ind)

In [40]:
import wget
for u in ['ИМСИТ', 'БашГУ', 'БГУ Петровского']:
    path = 'files/' + u + '/'
    tmp_df = courses_df[courses_df['ВУЗ'] == u]
    for _, row in tmp_df.iterrows():
        path_ = path + row['Факультет'] + '_' + row['ОП'] + '/'
        print(row['ОП'] )
        wget.download(row['Ссылка'], path_ + row['Дисциплина'] + '.pdf')
        break
    break

# Загрузка данных

In [41]:
import os
import wget
for ind, row in df[df['Тип']=='обязательный'].iterrows():
    path = 'files/' + row['ВУЗ'] + '/' + row['Факультет'] + '/' + row['ОП'] + '/' + row['Курс'] + '/'
    try:
        os.makedirs(path)
    except:
        pass
    try:
        wget.download(row['Ссылка'], path + str(ind) + ' ' + row['Дисциплина'] + '.pdf')
    except:
        print('Failed for ', ind)